In [2]:
# 初始化 PDF文件路径
pdf_path = "../../90-文档_Data/复杂PDF/billionaires_page-1-5.pdf"

#### 1. camelot 提取PDF表格

In [3]:
import camelot
# import pandas as pd
# from ctypes.util import find_library
# find_library("gs")
import time

start_time = time.time()
tables = camelot.read_pdf(pdf_path, pages="all")
end_time = time.time()
print(f"PDF表格解析耗时: {end_time - start_time:.2f}秒")

# 转换所有表格为 DataFrame
if tables:
    # 遍历所有表格
    for i, table in enumerate(tables, 1):
        # 将表格转换为 DataFrame
        df = table.df
        
        # 打印当前表格数据
        print(f"\n表格 {i} 数据:")
        print(df)
        
        # 显示基本信息
        print(f"\n表格 {i} 基本信息:")
        print(df.info())
        
        # 保存到CSV文件
        csv_filename = f"billionaires_table_{i}.csv"
        df.to_csv(csv_filename, index=False)
        print(f"\n表格 {i} 数据已保存到 {csv_filename}")


PDF表格解析耗时: 4.08秒

表格 1 数据:
      0                                           1
0  Icon                                 Description
1        Has not changed from the previous ranking.
2          Has increased from the previous ranking.
3          Has decreased from the previous ranking.

表格 1 基本信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       4 non-null      object
 1   1       4 non-null      object
dtypes: object(2)
memory usage: 196.0+ bytes
None

表格 1 数据已保存到 billionaires_table_1.csv

表格 2 数据:
      0                          1                 2    3               4  \
0   No.                       Name  Net worth\n(USD)  Age     Nationality   
1     1  Bernard Arnault &\nfamily      $211 billion   74          France   
2     2                  Elon Musk      $180 billion   51  United\nStates   
3     3                 Jeff Bezos      $114 billio

#### 2. pdfplumber 提取PDF表格

In [3]:
import pdfplumber
import pandas as pd
import time

# 记录开始时间
start_time = time.time()

# 打开PDF文件
pdf = pdfplumber.open(pdf_path)

# 遍历每一页
for page in pdf.pages:
    # 提取表格
    tables = page.extract_tables()
    
    # 检查是否找到表格
    if tables:
        print(f"在第 {page.page_number} 页找到 {len(tables)} 个表格")
        
        # 遍历该页的所有表格
        for i, table in enumerate(tables):
            print(f"\n处理第 {i+1} 个表格:")
            
            # 将表格转换为DataFrame
            df = pd.DataFrame(table)
            
            # 如果第一行包含列名，可以设置为列名
            if len(df) > 0:
                df.columns = df.iloc[0]
                df = df.iloc[1:]  # 删除重复的列名行
                
            print(df)
            print("-" * 50)

# 关闭PDF
pdf.close()

# 记录结束时间并计算总耗时
end_time = time.time()
print(f"\nPDF表格提取总耗时: {end_time - start_time:.2f}秒")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


在第 1 页找到 1 个表格

处理第 1 个表格:
0  List of the world's billionaires, ranked in order of net worth
1   The net worth of the world's billionaires incr...            
2                                 Publication details            
3    Publisher Whale Media Investments\nForbes family            
4                                  Publication Forbes            
5                       First published March 1987[1]            
6                    Latest publication April 4, 2023            
7                      Current list details (2023)[2]            
8                          Wealthiest Bernard Arnault            
9                      Net worth (1st) US$211 billion            
10          Number of 2,640 (from 2668)\nbillionaires            
11  Total list net worth US$12.2 trillion (from US...            
12                                Number of women 337            
13                       New members to the 150\nlist            
14  Forbes: The World's Billionaires website (htt

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


在第 5 页找到 2 个表格

处理第 1 个表格:
0  No.                      Name Net worth (USD) Age    Nationality  \
1    1                Jeff Bezos    $113 billion  56  United States   
2    2                Bill Gates     $98 billion  64  United States   
3    3  Bernard Arnault & family     $76 billion  71         France   
4    4            Warren Buffett   $67.5 billion  89  United States   
5    5             Larry Ellison     $59 billion  75  United States   
6    6            Amancio Ortega   $55.1 billion  84          Spain   
7    7           Mark Zuckerberg   $54.7 billion  35  United States   
8    8                Jim Walton   $54.6 billion  71  United States   
9    9              Alice Walton   $54.4 billion  70  United States   
10  10          S. Robson Walton   $54.1 billion  77  United States   

0  Source(s) of wealth  
1               Amazon  
2            Microsoft  
3                 LVMH  
4   Berkshire Hathaway  
5   Oracle Corporation  
6        Inditex, Zara  
7       Facebook

#### 3. pdfplumber提取PDF表格并问答

In [5]:
import pdfplumber
import pandas as pd
from llama_index.core import VectorStoreIndex
from llama_index.core import Document
from typing import List

# 打开 PDF 并解析表格
with pdfplumber.open(pdf_path) as pdf:
    tables = []
    for page in pdf.pages:
        table = page.extract_table()
        if table:
            tables.append(table)

# 转换所有表格为 DataFrame 并构建文档
documents: List[Document] = []
if tables:
    # 遍历所有表格
    for i, table in enumerate(tables, 1):
        # 将表格转换为 DataFrame
        df = pd.DataFrame(table)
        
        # 保存到CSV文件
        # csv_filename = f"billionaires_table_{i}.csv"
        # df.to_csv(csv_filename, index=False)
        # print(f"\n表格 {i} 数据已保存到 {csv_filename}")
        
        # 将DataFrame转换为文本
        text = df.to_string()
        
        # 创建Document对象
        doc = Document(text=text, metadata={"source": f"表格{i}"})
        documents.append(doc)

# 构建索引
index = VectorStoreIndex.from_documents(documents)

# 创建查询引擎
query_engine = index.as_query_engine()

# 示例问答
questions = [
    "2023年谁是最富有的人?",
    "最年轻的富豪是谁?"
]

print("\n===== 问答演示 =====")
for question in questions:
    response = query_engine.query(question)
    print(f"\n问题: {question}")
    print(f"回答: {response}")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox



===== 问答演示 =====

问题: 2023年谁是最富有的人?
回答: Elon Musk.

问题: 最年轻的富豪是谁?
回答: Mark Zuckerberg
